In [1]:
import json
import pandas as pd
import urllib.parse
from collections import Counter
import numpy as np

In [43]:
### PARAMETERS
# -------------

jsonPath = "C:/Users/agrog/Documents/Oxford/analysis/effugium/studies/jan24Pilot/oxfordRoomworld24.json"
csvPath = "C:/Users/agrog/Documents/Oxford/analysis/effugium/studies/jan24Pilot/roomworldPilotRounds.csv"
# -------------

In [5]:
with open(jsonPath, 'rb') as h:
    qs = json.load(h)

data = [i['fields'] for i in qs]

In [15]:
dataDf = pd.DataFrame.from_dict(data)
dataDf.head()

,UserId,iv,tag,data,gameParameters,riseTracking,registeredComplete,playMode,timeCreated,lastModified,cohortYear,isStaff,environment
0,alex,None,None,"[{'resp': {'tool': ['none', 'skip'], 'xloc': [...","{'c_order': [15, 12, 13, 3, 7, 4, 14, 11, 17, ...","[nothing, ok, hi]",False,4D,2023-11-20T15:11:33.592Z,2024-01-19T11:29:13.978Z,oxfordNov24,False,production
1,mira,None,None,"[{'resp': {'tool': ['none', 'k2', 'skip'], 'xl...","{'c_order': [17, 6, 7, 5, 1, 16, 8, 11, 4, 14,...","{'roundNum': 0, 'maxRounds': 92, 'timesComplet...",False,4D,2023-11-27T14:10:27.538Z,2023-12-04T12:13:37.546Z,test,False,production
2,489606266,None,None,{},"{'c_order': [11, 16, 3, 2, 18, 9, 4, 8, 17, 1,...",{},False,4D,2023-12-01T12:38:53.657Z,2023-12-01T12:38:53.657Z,oxford24,False,production
3,51710212,None,None,"[{'resp': {'tool': ['none', 'k11', 'k2', 'none...","{'c_order': [10, 13, 14, 12, 11, 6, 7, 5, 18, ...","{'roundNum': 2, 'maxRounds': 92, 'timesComplet...",False,4D,2023-12-07T14:36:40.370Z,2023-12-07T14:36:58.577Z,oxford24,False,production
4,1359218913],None,None,{},"{'c_order': [10, 12, 4, 6, 16, 5, 13, 2, 1, 3,...",{},False,4D,2023-12-09T20:22:07.233Z,2023-12-09T20:22:07.233Z,oxford24,False,production


In [14]:
df = dataDf.loc[dataDf["UserId"].isin([
    "683539231",
    "1012720441",
    "584275331",
    "777757392",
    "677946104"
])]

In [68]:
def getRounds(df, save=False):
    # Expects a filtered dataframe from server json
    # Convert back to a dict to operate on
    data = pd.DataFrame.to_dict(df, orient="index")
    # Iterate over each participant
    roundLevelData = []
    for user in data:
        common = {'userId' : data[user]['UserId']}
        for attempt in data[user]['data']:
            tmp = {**common, **attempt}
            # The end game looping thing at the end was also spamming final round requests, so we need to remove these duplicates
            if len(roundLevelData) != 0 and roundLevelData[-1]["roundIndex"] == 94 and attempt["roundIndex"] == 94:
                pass
            else:
                roundLevelData.append(tmp)

    if save:
        rldf = pd.DataFrame.from_dict(roundLevelData)
        rldf.to_csv(csvPath, index=False)
    
    return roundLevelData

In [62]:
rld = getRounds(df, save=False)
# create dataframe
rldf = pd.DataFrame.from_dict(rld)

In [69]:
# Some summary statistics
accs = rldf.groupby(by="userId")["completed"].mean()
print(accs)
print(f"Mean accuracy: {np.mean(list(accs))}%")